In [3]:
#Import the required libraries 
import os
import pickle
import cv2
import mediapipe as mp
import numpy as np

# Suppress TensorFlow logging messages 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

2024-03-11 18:01:21.647633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model_dict = pickle.load(open('/Users/MkupeteJaah/Desktop/ProjectData/Model/model.p', 'rb'))
model = model_dict['model']

# Import the required modules from the mediapipe library
mp_hands = mp.solutions.hands          # Hands module for hand tracking

# Drawing utilities for rendering landmarks on the image
mp_drawing = mp.solutions.drawing_utils

# Drawing styles for configuring the appearance of landmarks
mp_drawing_styles = mp.solutions.drawing_styles

# Initialize the Hands module from mediapipe
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Suppress TensorFlow logging messages 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [5]:
#crate a labels-Alphabets dictinary 
labels_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 
                11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 
                21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
#Create function to track hands and translate the sign language to Alphabet
def MonitorAndPredSignLang():
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        data_aux = []
        x_ = []
        y_ = []
    
        ret, frame = cap.read()
    
        H, W, _ = frame.shape
    
        # Convert the BGR frame to RGB format
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
        results = hands.process(frame_rgb)
        # Check if there are multi hand landmarks detected in the frame
        if results.multi_hand_landmarks:
            # Iterate through each set of hand landmarks
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw landmarks and connections on the frame
                mp_drawing.draw_landmarks(
                    frame,  # image to draw
                    hand_landmarks,  # model output
                    mp_hands.HAND_CONNECTIONS,  # hand connections
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

            # Loop through each landmark in the hand
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    # Append x and y coordinates to lists for further processing
                    x_.append(x)
                    y_.append(y)

                # Loop through each landmark in the hand again
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    # Calculate relative positions and append to data_aux
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))
                    
            # Calculate bounding box coordinates based on normalized hand landmarks
            x1 = int(min(x_) * W) - 10
            y1 = int(min(y_) * H) - 10
    
            x2 = int(max(x_) * W) - 10
            y2 = int(max(y_) * H) - 10
    
            # Make a prediction using the saved machine learning model
            if np.asarray(data_aux).size == 42:
                prediction = model.predict([np.asarray(data_aux)])
            
    
            # Get the predicted character from the labels dictionary
            predicted_character = labels_dict[int(prediction[0])]
    
            # Draw a rectangle around the detected hand
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)

            # Display the predicted character near the hand
            cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                        cv2.LINE_AA)
    
        cv2.imshow('frame', frame)
        cv2.waitKey(1)

In [7]:
cap = cv2.VideoCapture(0)
MonitorAndPredSignLang()
cap.release()
cv2.destroyAllWindows()

2024-03-11 18:01:29.977 Python[11409:737625] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
